<a href="https://colab.research.google.com/github/gitkundan/mle/blob/main/Project_Template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# New section

High Level Steps:

1. Import Modules,set environment
2. Load dataset
3. 

2.   List item


1.   List item
2.   List item





In [16]:
#1. Import Modules, set environment
# a) Load libraries
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import missingno as msno
import ssl 
import certifi
from sklearn.datasets import fetch_openml 


In [17]:
# 2. Load Dataset
ssl._create_default_https_context = ssl._create_unverified_context
# Download the dataset from openml
# dataset = fetch_openml(data_id=42803, as_frame=True) #Accident
dataset = fetch_openml(data_id=43454, as_frame=True) #Credit risk

#Extract feature matrix X and show 5 random samples
df = dataset["frame"]
df.sample(5)


/usr/local/lib/python3.8/dist-packages/sklearn/datasets/_openml.py:932: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_status,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length
2244,25,31000,MORTGAGE,4.0,VENTURE,B,10000,10.38,0,0.32,N,4
7655,24,47480,RENT,8.0,MEDICAL,B,6900,9.88,0,0.15,N,2
21710,34,50000,RENT,5.0,PERSONAL,B,6000,9.91,0,0.12,N,8
16000,26,42000,RENT,2.0,EDUCATION,C,4800,14.27,1,0.11,Y,2
25499,29,85000,MORTGAGE,2.0,HOMEIMPROVEMENT,A,10000,7.88,0,0.12,N,10


In [18]:
"""
3. Structural Analysis:
(a) basic queries : sample, info, describe, shape
(b) Cast to correct data types
e.g. df['measles_'] = df['measles_'].astype('int64')
(c) Treat obviously inocorrect values

# If you have values in your columns that you want to replace - use this for loop
# For instance - in the CreditScoring dataset - there are numerous 99999999 that need to be replaced

# for c in ['income', 'assets', 'debt']:
#    df[c] = df[c].replace(to_replace=99999999, value=np.nan)


# If you want to remove a value from a column - use this: 
# df = df[df.status != 'unk']   # This removes the value 'unk' from your data in the column.  Modify as needed

# drop multiple columns by name
# df.drop(['column_name1', 'column_name2'], axis=1, inplace=True)

pd.value_counts(df.dtypes)

df.nunique().to_frame()
#identify what is categorical
# identify too many value in categorical - OHE will turn into columns

""" 

"\n3. Structural Analysis:\n(a) basic queries : sample, info, describe, shape\n(b) Cast to correct data types\ne.g. df['measles_'] = df['measles_'].astype('int64')\n(c) Treat obviously inocorrect values\n\n# If you have values in your columns that you want to replace - use this for loop\n# For instance - in the CreditScoring dataset - there are numerous 99999999 that need to be replaced\n\n# for c in ['income', 'assets', 'debt']:\n#    df[c] = df[c].replace(to_replace=99999999, value=np.nan)\n\n\n# If you want to remove a value from a column - use this: \n# df = df[df.status != 'unk']   # This removes the value 'unk' from your data in the column.  Modify as needed\n\n# drop multiple columns by name\n# df.drop(['column_name1', 'column_name2'], axis=1, inplace=True)\n\npd.value_counts(df.dtypes)\n\ndf.nunique().to_frame()\n#identify what is categorical\n# identify too many value in categorical - OHE will turn into columns\n\n"

In [19]:
# 3. Qualitative Investigation of the Data
# a) Duplicates
# Duplicates in the Columns?
df.duplicated()
df.duplicated().sum()

# Duplicated Rows?
df[df.duplicated()]
# Find duplicates in specific columns from your dataset.  Replace with your columns obviously.  keep the first or last dupe.
# df.loc[df.duplicated(keep='last'), ['InvoiceNo', 'StockCode', 'InvoiceDate', 'CustomerID']]

# Only consider duplicates in these columns and drop only them
# df.duplicated(subset=['InvoiceNo', 'StockCode', 'InvoiceDate','CustomerID'], keep='first').sum()
# By looking only at these four columns instead of all of them, we can see that the number of duplicate rows may increase/decrease
# This means that there are rows that have the exact same values as these four columns but have different values in 
# other columns, which means they may be different records. 
# In most cases, it is better to use all the columns to identify duplicate records.

# df_unique = df.drop_duplicates(keep='first')
# df.drop_duplicates(keep='first')

# Check number of duplicates while ignoring the index feature
n_duplicates = df.drop(labels=['accident_index'], axis=1).duplicated().sum()

print(f"You seem to have {n_duplicates} duplicates in your database.")

#  Extract column names of all features, except 'Accident_Index'
columns_to_consider = df.drop(labels=['accident_index'], axis=1).columns

# Drop duplicates based on 'columns_to_consider'
df.drop_duplicates(subset=columns_to_consider, inplace=True)
df.shape

KeyError: ignored

In [ ]:
# MISSING VALUES
df.isna()
df.isna().sum()

# Check for missing values in a single column
# df[df['Description'].isna()]

# List all rows that are missing a value in this field
# df.dropna(subset=['Description'])  

# Drop all rows that are missing a value in this field:
# df.dropna(subset=['Description'], inplace=True)

# b) Missing Values per Sample (Big Holes)


plt.figure(figsize=(15, 8))
sns.set_style('whitegrid')

g = sns.heatmap(df.isnull(), cbar=False, cmap='viridis')
# g = sns.heatmap(df_X.loc[df_X.isnull().sum(1).sort_values(ascending=1).index].isnull(), cbar=False, cmap='viridis')
g.set_xlabel('Column Number')
g.set_ylabel('Sample Number')

#Decision to be done first with columns : which columns to drop
#Then next look at rows and see which rows to drop

msno.matrix(df, labels=True, sort='descending', color=(0.27, 0.52, 1.0));

#Objective is to elimiate large holes i.e. drop columns which have lots of missing values
#Good idea maybe to keep all rows and columns and get a baseline model working. Iterate later to drop rows and columns and check the model performance again

In [ ]:
# Make a decision... drop rows that are 20% or more empty (you set the threshhold)
df = df.dropna(thresh=df.shape[1] * 0.80, axis = 0).reset_index(drop=True)
df.shape

In [ ]:
# c) Missing Values per Feature (Big Holes)
df.isna().mean().sort_values().plot(
    kind="bar", figsize=(15, 4),
    title="Percentage of missing values per feature",
    ylabel="Ratio of missing values per feature");

In [ ]:
# drop any col that is more than 15% empty
df = df.dropna(thresh=df.shape[0] * 0.85,axis=1)
df.shape

In [ ]:
# d) Impute Values (Small Holes)
# df['CustomerID'].fillna('Missing', inplace=True)

# Replace NaN one column with the median
# df['col1'] = df['col1'].fillna(df['col1'].median())
# df = df.fillna(df.median())

# to see categorical encoding - see:  https://github.com/fenago/eda/blob/main/Cars_XGBoost.ipynb

In [ ]:
# 4. Content Investigation of the Data
# a) Distributions of data in each feature
# Plots the histogram for each numerical feature in a separate subplot
df.hist(bins=25, figsize=(15, 25), layout=(-1, 5), edgecolor="black")
plt.tight_layout();

#look at columns that are boolean, have only one value

In [ ]:
# b) Patterns
# Creates mask to identify numerical features with more or less than 25 unique features
cols_continuous = df.select_dtypes(include="number").nunique() >= 25

# Create a new dataframe which only contains the continuous features
df_continuous = df[cols_continuous[cols_continuous].index]
df_continuous.shape

sns.pairplot(df_continuous, height=1.5, plot_kws={"s": 2, "alpha": 0.2});


In [ ]:
# Create a new dataframe which doesn't contain the numerical continuous features
df_discrete = df[cols_continuous[~cols_continuous].index]
df_discrete.shape

In [ ]:
# Feature Relationships
# Evaluate but remember to consider multicollinearity

# Computes feature correlation
df_corr = df.corr(method="spearman") # pearson assumes a linear relationship... spearman does not

# Create labels for the correlation matrix
labels = np.where(np.abs(df_corr)>0.75, "S",
                  np.where(np.abs(df_corr)>0.5, "M",
                           np.where(np.abs(df_corr)>0.25, "W", "")))

# Plot correlation matrix
plt.figure(figsize=(15, 15))
sns.heatmap(df_corr, mask=np.eye(len(df_corr)), square=True,
            center=0, annot=labels, fmt='', linewidths=.5,
            cmap="vlag", cbar_kws={"shrink": 0.8});

In [ ]:
#  Creates a mask to remove the diagonal and the upper triangle.
lower_triangle_mask = np.tril(np.ones(df_corr.shape), k=-1).astype("bool")

#  Stack all correlations, after applying the mask
df_corr_stacked = df_corr.where(lower_triangle_mask).stack().sort_values()

#  Showing the lowest and highest correlations in the correlation matrix
display(df_corr_stacked)

In [ ]:
# # c) Split into Numeric List and Categorical List
# # Split the data into numeric and categorical lists and dataframes
# numerics = ['int16','int32','int64','float64']
# catDF = df.select_dtypes(exclude=numerics) #THESE WOULD BE OHE LATER
# numDF = df.select_dtypes(include=numerics) #THESE WOULD BE NORMALIZE/STANDARDIZE LATER
# catDF.head()
# numDF.head()
# # Display numerical features --> numeric features
# df.select_dtypes(include="number").head()